# __IBM Applied Data Science Capstone Course by Coursera__

### Week 3 Part 1
#### Build a dataframe of postal code of each neighborhood along with the borough name and neighborhood name in Toronto.

In [40]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.0.3)


#### 1. Import Libraries

In [41]:
get_ipython().system(u' pip install beautifulsoup4')

In [42]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files



import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print("Libraries imported.")

Libraries imported.


In [43]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### 2. Scrap data from Wikipedia page into dataframe 

In [44]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [45]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [46]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

#### Using BeautifulSoup

In [52]:
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    
    # append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [53]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df['PostalCode'] = toronto_df['PostalCode'].replace('\n','', regex=True)
toronto_df['Borough'] = toronto_df['Borough'].replace('\n','', regex=True)
toronto_df.head()

ImportError: cannot import name 'is_url'

  PostalCode           Borough                Neighborhood
0        M1A      Not assigned                            
1        M2A      Not assigned                            
2        M3A        North York                   Parkwoods
3        M4A        North York            Victoria Village
4        M5A  Downtown Toronto  Regent Park / Harbourfront

#### 3. Drop cells with a Borough that is "Not assigned

In [23]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

ImportError: cannot import name 'is_url'

  PostalCode           Borough                                  Neighborhood
0        M3A        North York                                     Parkwoods
1        M4A        North York                              Victoria Village
2        M5A  Downtown Toronto                    Regent Park / Harbourfront
3        M6A        North York             Lawrence Manor / Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government

In [ ]:
#### 4. Get Neighborhood in the same borough

In [37]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

ImportError: cannot import name 'is_url'

  PostalCode      Borough                                       Neighborhood
0        M1B  Scarborough                   Malvern / Rouge, Malvern / Rouge
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek, Roug...
2        M1E  Scarborough  Guildwood / Morningside / West Hill, Guildwood...
3        M1G  Scarborough                                     Woburn, Woburn
4        M1H  Scarborough                               Cedarbrae, Cedarbrae

#### 5. For the Neighborhood which is "Not assigned" make the value same as Borough

In [38]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

ImportError: cannot import name 'is_url'

  PostalCode      Borough                                       Neighborhood
0        M1B  Scarborough                   Malvern / Rouge, Malvern / Rouge
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek, Roug...
2        M1E  Scarborough  Guildwood / Morningside / West Hill, Guildwood...
3        M1G  Scarborough                                     Woburn, Woburn
4        M1H  Scarborough                               Cedarbrae, Cedarbrae

#### 6.Check whether it is the same as required by the question

In [39]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

ImportError: cannot import name 'is_url'

   PostalCode           Borough  \
0         M5G  Downtown Toronto   
1         M2H        North York   
2         M4B         East York   
3         M1J       Scarborough   
4         M4G         East York   
5         M4M      East Toronto   
6         M1R       Scarborough   
7         M9V         Etobicoke   
8         M9L        North York   
9         M5V  Downtown Toronto   
10        M1B       Scarborough   
11        M5A  Downtown Toronto   

                                         Neighborhood  
0              Central Bay Street, Central Bay Street  
1                Hillcrest Village, Hillcrest Village  
2   Parkview Hill / Woodbine Gardens, Parkview Hil...  
3            Scarborough Village, Scarborough Village  
4                                    Leaside, Leaside  
5                    Studio District, Studio District  
6              Wexford / Maryvale, Wexford / Maryvale  
7   South Steeles / Silverstone / Humbergate / Jam...  
8                        Humber Summit, 

#### 7. Finally, print the number of the rows of cleaned dataframe

In [21]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)